In [ ]:
from functools import cache

EXAMPLE = "../example.txt"
INPUT = "../input.txt"

In [ ]:
def get_codes(input_file_name):
    codes = []
    with open(input_file_name, 'r') as f:
        for line in f:
            codes.append(line.strip().replace("\n", ""))
    return codes

In [ ]:
codes = get_codes(EXAMPLE)
print(codes)

In [ ]:
DIGITS = {'A': (3, 2), '0': (3, 1), '1': (2, 0), '2': (2, 1), '3': (2, 2), '4': (1, 0), '5': (1, 1), '6': (1, 2), '7': (0, 0), '8': (0, 1), '9': (0, 2), 'FORBIDDEN': (3, 0)}
ARROWS = {'A': (0, 2), '^': (0, 1), '<': (1, 0), 'v': (1, 1), '>': (1, 2), 'FORBIDDEN': (0, 0)}

In [ ]:
def get_arrow_moves_between_entities(e1, e2, desc):
    vertical_diff = desc[e2][0] - desc[e1][0]
    horizontal_diff = desc[e2][1] - desc[e1][1]
    nb_of_vertical_moves = abs(vertical_diff)
    nb_of_horizontal_moves = abs(horizontal_diff)
    moves = {'^': 0, '<': 0, 'v': 0, '>': 0}
    if vertical_diff <= 0:
        moves['^'] = nb_of_vertical_moves
    else:
        moves['v'] = nb_of_vertical_moves
    if horizontal_diff >= 0:
        moves['>'] = nb_of_horizontal_moves
    else:
        moves['<'] = nb_of_horizontal_moves
    return moves

In [ ]:
@cache
def get_arrow_moves_between_digits(d1, d2):
    return get_arrow_moves_between_entities(d1, d2, DIGITS)

@cache
def get_arrow_moves_between_arrows(a1, a2):
    return get_arrow_moves_between_entities(a1, a2, ARROWS)

In [ ]:
print("0 to 9: ", get_arrow_moves_between_digits('0', '9'))
print("7 to 2: ", get_arrow_moves_between_digits('7', '2'))
print("A to <: ", get_arrow_moves_between_arrows('A', '<'))
print("^ to >: ", get_arrow_moves_between_arrows('^', '>'))

In [ ]:
def get_possible_sequences(arrow_moves: dict) -> list[str]:
    sequences = []
    if not any([nb > 0 for nb in arrow_moves.values()]):
        return [""]
    for arrow, nb in arrow_moves.items():
        if nb > 0:
            new_arrow_moves = arrow_moves.copy()
            new_arrow_moves[arrow] -= 1
            sequences += [arrow + tail for tail in get_possible_sequences(new_arrow_moves)]
    return sequences

In [ ]:
print("0 to 9: ", get_arrow_moves_between_digits('0', '9'))
print(get_possible_sequences(get_arrow_moves_between_digits('0', '9')))
print("7 to 2: ", get_arrow_moves_between_digits('7', '2'))
print(get_possible_sequences(get_arrow_moves_between_digits('7', '2')))
print("3 to 7: ", get_arrow_moves_between_digits('3', '7'))
print(get_possible_sequences(get_arrow_moves_between_digits('3', '7')))

In [ ]:
ARROW_OFFSET = {'^': (-1, 0), '<': (0, -1), 'v': (1, 0), '>': (0, 1)}

Find all the sequences of buttons to press on the n+1 level keypad in order to go from one place to another and press a button on the n level keypad

In [ ]:
def get_possible_sequences_between_entities(e1, e2, desc):
    arrow_moves = get_arrow_moves_between_entities(e1, e2, desc)
    sequences = get_possible_sequences(arrow_moves)
    possible_moves = []
    start_row, start_col = desc[e1]
    end_row, end_col = desc[e2]
    for sequence in sequences:
        row, col = start_row, start_col
        for arrow in sequence:
            r, c = row + ARROW_OFFSET[arrow][0], col + ARROW_OFFSET[arrow][1]
            if (r, c) == desc["FORBIDDEN"]:
                break
            row, col = r, c
        if (row, col) == (end_row, end_col):
            possible_moves.append(sequence + 'A')
    return possible_moves

In [ ]:
@cache
def get_possible_sequences_between_digits(d1, d2):
    return get_possible_sequences_between_entities(d1, d2, DIGITS)

@cache
def get_possible_sequences_between_arrows(a1, a2):
    return get_possible_sequences_between_entities(a1, a2, ARROWS)

In [ ]:
print(get_possible_sequences_between_digits('0', '9'))
print(get_possible_sequences_between_digits('7', 'A'))
print(get_possible_sequences_between_arrows('A', '<'))

In [ ]:
def get_level1_sequences_for_level0_sequence(l0_seq):
    level_1_sequences = ['']
    i = 0
    while i < len(l0_seq) - 1:
        d1, d2 = l0_seq[i], l0_seq[i + 1]
        l1_seqs = get_possible_sequences_between_entities(d1, d2, DIGITS)
        new_l1_sequences = []
        for current_l1_seq in level_1_sequences:
            for l1_seq in l1_seqs:
                new_l1_sequences.append(current_l1_seq + l1_seq)
        level_1_sequences = new_l1_sequences
        i += 1
    return level_1_sequences

In [ ]:
print(get_level1_sequences_for_level0_sequence("A029A"))

In [ ]:
def get_next_level_sequences_for_sequence(seq, desc):
    next_level_sequences = ['']
    i = 0
    while i < len(seq) - 1:
        e1, e2 = seq[i], seq[i + 1]
        next_seqs = get_possible_sequences_between_entities(e1, e2, desc)
        new_sequences = []
        for current_seq in next_level_sequences:
            for next_seq in next_seqs:
                new_sequences.append(current_seq + next_seq)
        next_level_sequences = new_sequences
        i += 1
    return next_level_sequences

In [ ]:
@cache
def get_next_level_sequences_for_sequence_of_digits(seq):
    return get_next_level_sequences_for_sequence(seq, DIGITS)

@cache
def get_next_level_sequences_for_sequence_of_arrows(seq):
    return get_next_level_sequences_for_sequence(seq, ARROWS)

In [ ]:
print(get_next_level_sequences_for_sequence_of_digits("A0"))
print(get_next_level_sequences_for_sequence_of_arrows("A<A"))
print(get_next_level_sequences_for_sequence_of_arrows("Av<<A>>^A")) # Answer: <vA<AA>>^AvAA<^A>A

In [ ]:
def get_last_level_sequences_for_first_level_digits(d1, d2):
    seq = d1 + d2
    level1_sequences = get_next_level_sequences_for_sequence_of_digits(seq)
    level2_sequences = []
    for l1_seq in level1_sequences:
        level2_sequences.extend(get_next_level_sequences_for_sequence_of_arrows('A' + l1_seq))
    level3_sequences = []
    for l2_seq in level2_sequences:
        level3_sequences.extend(get_next_level_sequences_for_sequence_of_arrows('A' + l2_seq))
    return level3_sequences

In [ ]:
print(get_last_level_sequences_for_first_level_digits('A', '0'))
print(get_last_level_sequences_for_first_level_digits('0', '2'))
print(get_last_level_sequences_for_first_level_digits('2', '9'))
print(get_last_level_sequences_for_first_level_digits('9', 'A'))

In [ ]:
def get_min_length_for_code(code):
    code = 'A' + code
    result = 0
    i = 0
    while i < len(code) - 1:
        d1, d2 = code[i], code[i+1]
        seqs = get_last_level_sequences_for_first_level_digits(d1, d2)
        min_l = min([len(s) for s in seqs])
        result += min_l
        i+=1
    return result

In [ ]:
for code in codes:
    print(get_min_length_for_code(code))

In [ ]:
def part_1(input_file_name):
    codes = get_codes(input_file_name)
    result = 0
    for code in codes:
        number = int(code[:-1])
        length = get_min_length_for_code(code)
        complexity = number * length
        result += complexity
    print(result)

In [ ]:
part_1(EXAMPLE)

In [ ]:
part_1(INPUT)

In [ ]:
TRANSITIONS = {}
for a1 in ARROWS:
    for a2 in ARROWS:
        TRANSITIONS[(a1, a2)] = sum(get_arrow_moves_between_arrows(a1, a2).values())
print(TRANSITIONS)

In [ ]:
def transitions_from_seq(seq):
    transitions = {}
    i = 0
    while i < len(seq) - 1:
        transition = (seq[i], seq[i+1])
        if transition in transitions:
            transitions[transition] += 1
        else:
            transitions[transition] = 1
        i+=1
    return transitions

In [ ]:
def get_next_transitions_list(transition):
    (a0, a1) = transition
    sequences = get_possible_sequences_between_arrows(a0, a1)
    # min_score = -1
    # next_transitions = (-1, -1)
    transitions_list = []
    # print(len(sequences))
    for seq in sequences:
        transitions = transitions_from_seq('A' + seq)
        transitions_list.append(transitions)
        # score = sum([TRANSITIONS[t] for t in transitions])
        # if min_score == -1 or score < min_score:
        #     min_score = score
        #     next_transitions = transitions
    return transitions_list

Maybe I should just try all possible versions of NEXT_TRANSITIONS

In [ ]:
next_possible_transitions = []
for transition in TRANSITIONS:
    transitions_list = get_next_transitions_list(transition)
    new_dict_list = []
    if not transitions_list:
        continue
    for transitions in transitions_list:
        if not next_possible_transitions:
            new_dict = {}
            new_dict[transition] = transitions
            new_dict_list.append(new_dict)
        else:
            for next_transitions_dict in next_possible_transitions:
                new_dict = next_transitions_dict.copy()
                new_dict[transition] = transitions
                new_dict_list.append(new_dict)
    next_possible_transitions = new_dict_list
    # print(next_possible_transitions)

NEXT_TRANSITIONS = next_possible_transitions

# print(NEXT_TRANSITIONS)
print(len(NEXT_TRANSITIONS))
for transitions in NEXT_TRANSITIONS:
    print(transitions)

In [ ]:
def get_next_level(transitions, next_transitions):
    next_level_transitions = {}
    for transition, i in transitions.items():
        for next_transition, j in next_transitions[transition].items():
            if next_transition in next_level_transitions:
                next_level_transitions[next_transition] += i*j
            else:
                next_level_transitions[next_transition] = i*j
    return next_level_transitions

In [ ]:
min_l1 = -1
min_l2 = -1
for next_transitions in NEXT_TRANSITIONS:
    level_0 = transitions_from_seq("A<A^A>^^AvvvA")
    level_1 = get_next_level(level_0, next_transitions)
    length_1 = sum(level_1.values()) + 1
    if min_l1 == -1 or length_1 < min_l1:
        min_l1 = length_1
    level_2 = get_next_level(level_1, next_transitions)
    length_2 = sum(level_2.values()) + 1
    if min_l2 == -1 or length_2 < min_l2:
        min_l2 = length_2

print(min_l1)
print(len("Av<<A>>^A<A>AvA<^AA>A<vAAA>^A"))
print(min_l2)
print(len("A<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A"))

In [ ]:
def get_min_length_for_level_0_sequence(level_0_seq, nb_of_dir_keypads):
    min_l = -1
    for next_transitions in NEXT_TRANSITIONS:
        level = transitions_from_seq('A' + level_0_seq)
        for _ in range(nb_of_dir_keypads):
            level = get_next_level(level, next_transitions)
        length = sum(level.values())
        if min_l == -1 or length < min_l:
            min_l = length
    return min_l

In [ ]:
print(get_min_length_for_level_0_sequence("<A^A>^^AvvvA", 2))

In [ ]:
def get_level0_sequences_for_code(code):
    code = "A" + code
    i = 0
    sequences = []
    while i < len(code) - 1:
        d1, d2 = code[i], code[i + 1]
        seq = d1 + d2
        level_0_sequences = get_next_level_sequences_for_sequence_of_digits(seq)
        sequences.append(level_0_sequences)
        i += 1
    for i in range(1, len(sequences)):
        new_sequences = []
        for new_sequence in sequences[i]:
            for old_sequence in sequences[i - 1]:
                new_sequences.append(old_sequence + new_sequence)
        sequences[i] = new_sequences
    return sequences[-1]

In [ ]:
print(get_level0_sequences_for_code("029A"))

In [ ]:
def get_min_length_for_code(code, nb_of_dir_keypads):
    level_0_sequences = get_level0_sequences_for_code(code)
    lengths = []
    for level_0_seq in level_0_sequences:
        min_l = get_min_length_for_level_0_sequence(level_0_seq, nb_of_dir_keypads)
        lengths.append(min_l)
    return min(lengths)

In [ ]:
def part_2(input_file_name):
    codes = get_codes(input_file_name)
    result = 0
    for code in codes:
        number = int(code[:-1])
        length = get_min_length_for_code(code, 25)
        complexity = number * length
        result += complexity
    print(result)

In [ ]:
part_2(EXAMPLE)

In [ ]:
part_2(INPUT)